In [ ]:
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense,LSTM,Flatten,Dropout,Input,concatenate
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error,mean_squared_error,r2_score
import tensorflow as tf
from sklearn.model_selection import train_test_split

my_seed = 7
tf.compat.v1.set_random_seed(my_seed)
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.experimental.set_memory_growth(physical_devices[0],True)
#     assert tf.config.experimental.get_memory_growth(physical_devices[0])
# except:
#     pass

In [ ]:
data = pd.read_csv('data/banben2.csv',index_col='time')
data = data.iloc[:,[0,1,2,5,6,7]]

In [ ]:
data

In [ ]:
modility_num = data.iloc[:,[0,1,2]]
modility_ser = data['severity_num']
modility_pri = data['priority_num']

In [ ]:
modility_num.shape,modility_ser.shape,modility_pri.shape

In [ ]:
scaler1 = MinMaxScaler(feature_range=(0,1))
scaler2 = MinMaxScaler(feature_range=(0,1))
scaler3 = MinMaxScaler(feature_range=(0,1))
scaler4 = MinMaxScaler(feature_range=(0,1))
modility_num_norm = scaler1.fit_transform(modility_num)
modility_ser_norm = scaler2.fit_transform(modility_ser.values.reshape(-1,1))
modility_pri_norm = scaler3.fit_transform(modility_pri.values.reshape(-1,1))
number =  scaler4.fit_transform(data['number'].values.reshape(-1,1))

In [ ]:
# 这个才是对的！
def create_dataset(dataset,look_back):
    dataX,dataY = [],[]
#     注意：这个range里面到底要不要减一
    for i in range(len(dataset)-look_back):
        a = dataset[i:i+look_back]
        dataX.append(a)
        dataY.append(dataset[i+look_back,0])
    return np.array(dataX),np.array(dataY)

In [ ]:
test_size = 200
look_back = 7

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
dataY = data['category'][look_back:].values.reshape(-1,1)
dataY.shape

In [ ]:
one_hots = to_categorical(dataY-1)

In [ ]:
one_hots.shape

In [ ]:
dataX_all_1,dataY_all_1 = create_dataset(modility_num_norm,7)
dataX_all_2,dataY_all_2 = create_dataset(modility_ser_norm,7)
dataX_all_3,dataY_all_3 = create_dataset(modility_pri_norm,7)



trainX_1, testX_1,trainX_2, testX_2,trainX_3, testX_3, trainY, testY = train_test_split(dataX_all_1, dataX_all_2,dataX_all_3,one_hots, test_size=0.25, random_state=0)


# trainX_1 = dataX_all_1[:-test_size]
# testX_1 = dataX_all_1[-test_size:]
# trainX_2 = dataX_all_2[:-test_size]
# testX_2 = dataX_all_2[-test_size:]
# trainX_3 = dataX_all_3[:-test_size]
# testX_3 = dataX_all_3[-test_size:]
# trainY = dataY_all_1[:-test_size]
# testY = dataY_all_1[-test_size:]

In [ ]:
trainX_1.shape,trainX_2.shape,trainX_3.shape,trainY.shape

In [ ]:
number_input = Input(shape=(7,3),name = "number")
severity_input = Input(shape=(7,1),name = "severity")
priority_input = Input(shape=(7,1),name = "priority")
x1 = LSTM(64,return_sequences=False)(number_input)
x2 = LSTM(64,return_sequences=False)(severity_input)
x3 = LSTM(64,return_sequences=False)(priority_input)
x = concatenate([x1,x2,x3])
# x = Dense(64, name="dence")(x)
num_pred = Dense(3, activation = 'softmax',name="output")(x)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [ ]:
model = Model(inputs = [number_input,severity_input,priority_input],outputs = num_pred)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit({"number": trainX_1, "severity": trainX_2, "priority": trainX_3},{"output":trainY},epochs=100, batch_size=12, verbose=2,validation_split=0.2,callbacks=[callback])

In [ ]:
accuracy = model.evaluate([testX_1,testX_2,testX_3], testY, batch_size=12, verbose=2)

In [ ]:
testPredict = model.predict([testX_1,testX_2,testX_3])

In [ ]:
testPredict

In [ ]:
y_pred = np.argmax(testPredict, axis=1)

In [ ]:
y_pred

In [ ]:
y_real = np.argmax(testY, axis=1)

In [ ]:
y_real

In [ ]:
from sklearn.metrics import cohen_kappa_score
kappa = cohen_kappa_score(y_real,y_pred)
# kappa = cohen_kappa_score(dataY[-30:],y_pred)
#(label除非是你想计算其中的分类子集的kappa系数，否则不需要设置)
kappa

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_real,y_pred))
# print(classification_report(dataY[-30:],y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
con  = confusion_matrix(y_real,y_pred)
con

In [ ]:
# https://www.aiuai.cn/aifarm1335.html

In [ ]:
from sklearn.metrics import confusion_matrix

labels = ['low', 'middle','high']
cm = con
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

confusion = con
# 热度图，后面是指定的颜色块，可设置其他的不同颜色
plt.imshow(confusion, cmap=plt.cm.Blues)
# plt.imshow(confusion, cmap=plt.cm.Wistia)
# ticks 坐标轴的坐标点
# label 坐标轴标签说明
indices = range(len(confusion))
# 第一个是迭代对象，表示坐标的显示顺序，第二个参数是坐标轴显示列表
#plt.xticks(indices, [0, 1, 2])
#plt.yticks(indices, [0, 1, 2])
plt.xticks(indices, ['low', 'middle','high'])
plt.yticks(indices, ['low', 'middle','high'])

plt.colorbar()

plt.xlabel('预测值')
plt.ylabel('真实值')
plt.title('banben2')

# plt.rcParams两行是用于解决标签不能显示汉字的问题
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 显示数据
for first_index in range(len(confusion)):  #第几行
    for second_index in range(len(confusion[first_index])):  #第几列
        plt.text(first_index, second_index, confusion[first_index][second_index])
plt.show()

In [ ]:
help(plt.cm)